In [1]:
import json 
import pandas as pd

In [2]:
import os
from pathlib import Path

print("cwd:", Path.cwd())
print("files here:", os.listdir(Path.cwd()))

print("cwd:", os.getcwd())
print("files here:", os.listdir(os.getcwd()))
print("up one:", os.listdir(os.path.join(os.getcwd(), "..")))

cwd: c:\Users\Dee\root\Projects\dev\graphgeeks-lab\odsc-agentic-ai-summit-2025
files here: ['.env', '.git', '.gitignore', '.venv', 'create_dataset.py', 'data', 'demo', 'fhir_kuzu_db', 'format_json.py', 'process_fhir_data.ipynb', 'process_fhir_data.py', 'pyproject.toml', 'README.md', 'src', 'uv.lock']
cwd: c:\Users\Dee\root\Projects\dev\graphgeeks-lab\odsc-agentic-ai-summit-2025
files here: ['.env', '.git', '.gitignore', '.venv', 'create_dataset.py', 'data', 'demo', 'fhir_kuzu_db', 'format_json.py', 'process_fhir_data.ipynb', 'process_fhir_data.py', 'pyproject.toml', 'README.md', 'src', 'uv.lock']
up one: ['.venv', 'asknews', 'awesome-graph-universe', 'baml-for-graph-101', 'demo_db', 'graph-algorithms', 'graph-explorer', 'graphfaker', 'odsc-agentic-ai-summit-2025', 'project-proposals', 'roadmap', 'test']


In [3]:
# Load the JSON array
print("Loading data from ./data/fhir_v2.jsonl")
with open('./data/fhir_v2.jsonl', 'r') as f:
    bundles = json.load(f)


Loading data from ./data/fhir_v2.jsonl


In [4]:
records = []

for bundle in bundles:
    bundle_id = bundle.get('id')
    for entry in bundle.get("entry", []):
        r = entry.get("resource", {})
        rec = {
            "bundle_id": bundle_id,
            "resourceType": r.get("resourceType"),
            "resource_id": r.get("id"),
            **r
        }
        records.append(rec)



df = pd.json_normalize(records)

In [ ]:
# show all rows
pd.set_option("display.max_rows", None)
# show all columns
pd.set_option("display.max_columns", None)
# don’t truncate column contents
pd.set_option("display.max_colwidth", None)
# widen the “page” so it doesn’t wrap as much
pd.set_option("display.width", 1000)

# now display your DataFrame in full
from IPython.display import display

In [8]:
df.columns

Index(['bundle_id', 'resourceType', 'resource_id', 'id', 'name', 'telecom',
       'gender', 'birthDate', 'address', 'multipleBirthBoolean',
       'communication', 'maritalStatus.coding', 'maritalStatus.text', 'status',
       'type', 'participant', 'period', 'serviceProvider', 'class.system',
       'class.code', 'subject.reference', 'subject.display', 'clinicalStatus',
       'verificationStatus', 'code', 'onsetDateTime', 'recordedDate',
       'clinicalStatus.coding', 'verificationStatus.coding', 'code.coding',
       'code.text', 'category', 'effectiveDateTime', 'issued', 'valueQuantity',
       'occurrenceDateTime', 'primarySource', 'vaccineCode.coding',
       'vaccineCode.text', 'patient.reference', 'encounter.reference',
       'maritalStatus', 'deceasedDateTime', 'valueQuantity.value',
       'valueQuantity.unit', 'valueQuantity.system', 'valueQuantity.code',
       'component', 'valueCodeableConcept', 'criticality', 'performedPeriod',
       'reasonReference', 'active', 'rea

In [12]:
df['name'][2]

[{'use': 'official',
  'family': 'Bashirian',
  'given': ['Hobert', 'Armand'],
  'prefix': ['Mr.']}]

In [13]:
df['address'][2]

[{'line': ['767 Windler Well'],
  'city': 'Easthampton',
  'state': 'Massachusetts',
  'postalCode': '01027',
  'country': 'US'}]

In [20]:
df['maritalStatus'][2]

np.float64(nan)

In [32]:
df_exp = df.explode('name').explode('address').reset_index(drop=True)

In [35]:
name_cols = pd.json_normalize(df_exp["name"])
#    for 'address'
addr_cols = pd.json_normalize(df_exp["address"])

In [37]:
df_clean = (
    df_exp
    .drop(columns=["name", "address"])
    .join([name_cols.add_prefix("name."), addr_cols.add_prefix("address.")])
)

In [38]:
print(df_clean.columns.tolist())
print(df_clean.head())

['bundle_id', 'resourceType', 'resource_id', 'id', 'telecom', 'gender', 'birthDate', 'multipleBirthBoolean', 'communication', 'maritalStatus.coding', 'maritalStatus.text', 'status', 'type', 'participant', 'period', 'serviceProvider', 'class.system', 'class.code', 'subject.reference', 'subject.display', 'clinicalStatus', 'verificationStatus', 'code', 'onsetDateTime', 'recordedDate', 'clinicalStatus.coding', 'verificationStatus.coding', 'code.coding', 'code.text', 'category', 'effectiveDateTime', 'issued', 'valueQuantity', 'occurrenceDateTime', 'primarySource', 'vaccineCode.coding', 'vaccineCode.text', 'patient.reference', 'encounter.reference', 'maritalStatus', 'deceasedDateTime', 'valueQuantity.value', 'valueQuantity.unit', 'valueQuantity.system', 'valueQuantity.code', 'component', 'valueCodeableConcept', 'criticality', 'performedPeriod', 'reasonReference', 'active', 'reasonCode', 'abatementDateTime', 'occurrenceString', 'period.start', 'period.end', 'serviceProvider.reference', 'servi

In [52]:
def safe_join(x):
    if isinstance(x, list):
        return " ".join(x)
    return x


# apply to your two columns
df_clean["given_names"]    = df_clean["name.given"].apply(safe_join)
df_clean["street_address"] = df_clean["address.line"].apply(safe_join)

In [53]:
df_clean['given_names'].head(10)

0     Rudolf Nolan
1              NaN
2    Hobert Armand
3              NaN
4              NaN
5              NaN
6              NaN
7      Sonia María
8      Asa Sherman
9              NaN
Name: given_names, dtype: object

In [54]:
# 3) Build the full_address and full_name columns
df_clean["full_address"] = (
    df_clean["street_address"]
    + ", "
    + df_clean["address.postalCode"].str.strip()
    + " "
    + df_clean["address.city"].str.strip()
).str.replace(r"\s+,", ",", regex=True)  # tidy up stray spaces before commas

df_clean["full_name"] = (
    df_clean["given_names"]
    + " "
    + df_clean["name.family"].str.strip()
).str.replace(r"\s+", " ", regex=True)

In [55]:
df_clean["full_address"].head(10)

0         983 Gutmann Burg, 02111 Boston
1                                    NaN
2    767 Windler Well, 01027 Easthampton
3                                    NaN
4                                    NaN
5                                    NaN
6                                    NaN
7          130 Rempel Vale, 02111 Boston
8           698 Kunde Spur, 02026 Dedham
9                                    NaN
Name: full_address, dtype: object

In [57]:
counts = df_clean[["full_address","full_name"]].count()
print(counts)

full_address    2169
full_name       3298
dtype: int64


In [58]:
# 2) Individually, checking for non-null and non-empty strings
street_count = df_clean["full_address"].notna().sum()
given_count  = df_clean["full_name"].notna().sum()
print(f"full_address appears in {street_count} rows")
print(f"full_name appears in  {given_count} rows")

# 3) If you want to be sure you’re not counting blank strings:
street_count = df_clean["full_address"].str.strip().ne("").sum()
given_count  = df_clean["full_name"].str.strip().ne("").sum()

# 4) If you instead want the frequency of each distinct value:
print(df_clean["full_address"].value_counts().head())
print(df_clean["full_name"].value_counts().head())

full_address appears in 2169 rows
full_name appears in  3298 rows
full_address
881 Main Street, 01420 Fitchburg         11
200 MAY ST, 027035520 SOUTH ATTLEBORO    10
170 GOVERNORS AVE, 021763225 MELROSE      9
41 & 45 MALL ROAD, 019602901 PEABODY      9
365 EAST ST, 018761950 TEWKSBURY          8
Name: count, dtype: int64
full_name
Alysha Koch         6
Latoyia Lindgren    5
Houston Funk        5
Alex Satterfield    5
Bennett Rippin      5
Name: count, dtype: int64


In [59]:
# 1) Compute counts
addr_counts  = df_clean["full_address"].value_counts()
name_counts  = df_clean["full_name"].value_counts()

# 2) Identify the values that occur more than once
dupe_addrs = addr_counts[addr_counts > 1].index.tolist()
dupe_names = name_counts[name_counts > 1].index.tolist()

# 3a) Filter rows that share a duplicate address
df_addr_dups = df_clean[df_clean["full_address"].isin(dupe_addrs)]
print("Addresses seen >1 times:")
print(df_addr_dups["full_address"].value_counts().head())

# 3b) Filter rows that share a duplicate name
df_name_dups = df_clean[df_clean["full_name"].isin(dupe_names)]
print("\nNames seen >1 times:")
print(df_name_dups["full_name"].value_counts().head())

# 4) If you want rows where *both* name and address are duplicated together
df_both_dups = (
    df_clean
    .duplicated(subset=["full_name","full_address"], keep=False)
)
df_pair_dups = df_clean[df_both_dups]
print("\nRows where (full_name, full_address) pair is repeated:")
print(df_pair_dups[["full_name","full_address"]].drop_duplicates().head())


Addresses seen >1 times:
full_address
881 Main Street, 01420 Fitchburg         11
200 MAY ST, 027035520 SOUTH ATTLEBORO    10
41 & 45 MALL ROAD, 019602901 PEABODY      9
170 GOVERNORS AVE, 021763225 MELROSE      9
14 PROSPECT STREET, 015681431 UPTON       8
Name: count, dtype: int64

Names seen >1 times:
full_name
Alysha Koch         6
Bennett Rippin      5
Maryann Hintz       5
Carisa Kutch        5
Latoyia Lindgren    5
Name: count, dtype: int64

Rows where (full_name, full_address) pair is repeated:
         full_name                               full_address
1              NaN                                        NaN
26  Eldora Langosh                                        NaN
37        Wes King                                        NaN
74             NaN           881 Main Street, 01420 Fitchburg
83             NaN  8 SHIPWAY PL # C-8, 021294301 CHARLESTOWN


In [62]:
# exact match
addr = "881 Main Street, 01420 Fitchburg"
df_fitchburg = df_clean[df_clean["full_address"] == addr]

# print(df_fitchburg.shape)
# print(df_fitchburg)

df_fitchburg


,bundle_id,resourceType,resource_id,id,telecom,gender,birthDate,multipleBirthBoolean,communication,maritalStatus.coding,...,name.suffix,address.line,address.city,address.state,address.postalCode,address.country,given_names,street_address,full_address,full_name
74,1,Organization,1,1,None,NaN,NaN,NaN,NaN,NaN,...,NaN,881 Main Street,Fitchburg,MA,01420,US,NaN,881 Main Street,"881 Main Street, 01420 Fitchburg",NaN
684,1,Practitioner,1,1,None,male,NaN,NaN,NaN,NaN,...,NaN,881 Main Street,Fitchburg,MA,01420,US,NaN,881 Main Street,"881 Main Street, 01420 Fitchburg",NaN
1459,1,Practitioner,1,1,"[{'system': 'email', 'value': 'Ted955.Reilly98...",male,NaN,NaN,NaN,NaN,...,NaN,881 Main Street,Fitchburg,MA,01420,US,Ted,881 Main Street,"881 Main Street, 01420 Fitchburg",Ted Reilly
1460,1,Organization,1,1,None,NaN,NaN,NaN,NaN,NaN,...,NaN,881 Main Street,Fitchburg,MA,01420,US,NaN,881 Main Street,"881 Main Street, 01420 Fitchburg",NaN
4678,1,Organization,1,1,"[{'system': 'phone', 'value': '978-342-9781 Or...",NaN,NaN,NaN,NaN,NaN,...,NaN,881 Main Street,Fitchburg,MA,01420,US,NaN,881 Main Street,"881 Main Street, 01420 Fitchburg",NaN
5043,1,Organization,1,1,"[{'system': 'phone', 'value': '978-342-9781 Or...",NaN,NaN,NaN,NaN,NaN,...,NaN,881 Main Street,Fitchburg,MA,01420,US,NaN,881 Main Street,"881 Main Street, 01420 Fitchburg",NaN
7127,1,Practitioner,1,1,"[{'system': 'email', 'value': 'Ted955.Reilly98...",male,NaN,NaN,NaN,NaN,...,NaN,881 Main Street,Fitchburg,MA,01420,US,Ted,881 Main Street,"881 Main Street, 01420 Fitchburg",Ted Reilly
7309,1,Organization,1,1,"[{'system': 'phone', 'value': '978-342-9781 Or...",NaN,NaN,NaN,NaN,NaN,...,NaN,881 Main Street,Fitchburg,MA,01420,US,NaN,881 Main Street,"881 Main Street, 01420 Fitchburg",NaN
7624,1,Practitioner,1,1,"[{'system': 'email', 'value': 'Ted955.Reilly98...",None,NaN,NaN,NaN,NaN,...,NaN,881 Main Street,Fitchburg,MA,01420,US,Ted,881 Main Street,"881 Main Street, 01420 Fitchburg",Ted Reilly
7625,1,Organization,1,1,None,NaN,NaN,NaN,NaN,NaN,...,NaN,881 Main Street,Fitchburg,MA,01420,US,NaN,881 Main Street,"881 Main Street, 01420 Fitchburg",NaN


In [64]:
# show all rows
pd.set_option("display.max_rows", None)
# show all columns
pd.set_option("display.max_columns", None)
# don’t truncate column contents
pd.set_option("display.max_colwidth", None)
# widen the “page” so it doesn’t wrap as much
pd.set_option("display.width", 1000)

# now display your DataFrame in full
from IPython.display import display
display(df_fitchburg)

,bundle_id,resourceType,resource_id,id,telecom,gender,birthDate,multipleBirthBoolean,communication,maritalStatus.coding,maritalStatus.text,status,type,participant,period,serviceProvider,class.system,class.code,subject.reference,subject.display,clinicalStatus,verificationStatus,code,onsetDateTime,recordedDate,clinicalStatus.coding,verificationStatus.coding,code.coding,code.text,category,effectiveDateTime,issued,valueQuantity,occurrenceDateTime,primarySource,vaccineCode.coding,vaccineCode.text,patient.reference,encounter.reference,maritalStatus,deceasedDateTime,valueQuantity.value,valueQuantity.unit,valueQuantity.system,valueQuantity.code,component,valueCodeableConcept,criticality,performedPeriod,reasonReference,active,reasonCode,abatementDateTime,occurrenceString,period.start,period.end,serviceProvider.reference,serviceProvider.display,performedPeriod.start,performedPeriod.end,reaction,multipleBirthInteger,valueCodeableConcept.coding,valueCodeableConcept.text,hospitalization,hospitalization.dischargeDisposition.coding,hospitalization.dischargeDisposition.text,valueString,name.use,name.family,name.given,name.prefix,name.suffix,address.line,address.city,address.state,address.postalCode,address.country,given_names,street_address,full_address,full_name
74,1,Organization,1,1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/organization-type', 'code': 'prov', 'display': 'Healthcare Provider'}], 'text': 'Healthcare Provider'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,881 Main Street,Fitchburg,MA,01420,US,NaN,881 Main Street,"881 Main Street, 01420 Fitchburg",NaN
684,1,Practitioner,1,1,None,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,881 Main Street,Fitchburg,MA,01420,US,NaN,881 Main Street,"881 Main Street, 01420 Fitchburg",NaN
1459,1,Practitioner,1,1,"[{'system': 'email', 'value': 'Ted955.Reilly981@example.com', 'use': 'work'}]",male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Reilly,Ted,[Dr.],NaN,881 Main Street,Fitchburg,MA,01420,US,Ted,881 Main Street,"881 Main Street, 01420 Fitchburg",Ted Reilly
1460,1,Organization,1,1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/organization-type', 'code': 'prov', 'display': 'Healthcare Provider'}], 'text': 'Healthcare Provider'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,881 Main Street,Fitchburg,MA,01420,US,NaN,881 Main Street,"881 Main Street, 01420 Fitchburg",NaN
4678,1,Organization,1,1,"[{'system': 'phone', 'value': '978-342-9781 Or 978-342-9781'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/organization-type', 'code': 'prov', 'display': 'Healthcare Provider'}], 'text': 'Healthcare Provider'}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,881 Main Street,Fitchburg,MA,01420,US,NaN,881 Main Street,"881 Main Street, 01420 Fitchburg",NaN
5043,1,Organization,1,1,"[{'system': 'phone', 'value': '978-342-9781 Or 978-342-9781'}]",NaN,NaN

In [78]:
# 0) Define the set of types to ignore
ignore_types = {"Practitioner", "Organization"}

# 1) Filter out those rows up front
df_patients = df_clean[~df_clean["resourceType"].isin(ignore_types)]

# 2) Compute address counts on that filtered set
addr_counts = df_patients["full_address"].value_counts()

# 3) Pick only the addresses seen more than once
dupe_addrs = addr_counts[addr_counts > 1].index.tolist()

# 4) Subset to just those duplicate addresses
df_addr_dups = df_patients[df_patients["full_address"].isin(dupe_addrs)]

# 5) Display the top ones
print("Addresses (excluding Practitioner & Organization) seen >1 times:")
print(df_addr_dups["full_address"].value_counts().head())


Addresses (excluding Practitioner & Organization) seen >1 times:
full_address
776 Bergstrom Wall, 02451 Waltham                   4
642 Glover Junction Suite 70, 01532 Northborough    4
549 Dare Pathway Unit 28, 00000 Medway              4
1040 Fay Corner Suite 97, 02128 Boston              4
431 Vandervort Neck, 01030 Agawam                   4
Name: count, dtype: int64


In [ ]:
# exact match
addr = "776 Bergstrom Wall, 02451 Waltham"
df_waltham = df_clean[df_clean["full_address"] == addr]


df_waltham

,bundle_id,resourceType,resource_id,id,telecom,gender,birthDate,multipleBirthBoolean,communication,maritalStatus.coding,maritalStatus.text,status,type,participant,period,serviceProvider,class.system,class.code,subject.reference,subject.display,clinicalStatus,verificationStatus,code,onsetDateTime,recordedDate,clinicalStatus.coding,verificationStatus.coding,code.coding,code.text,category,effectiveDateTime,issued,valueQuantity,occurrenceDateTime,primarySource,vaccineCode.coding,vaccineCode.text,patient.reference,encounter.reference,maritalStatus,deceasedDateTime,valueQuantity.value,valueQuantity.unit,valueQuantity.system,valueQuantity.code,component,valueCodeableConcept,criticality,performedPeriod,reasonReference,active,reasonCode,abatementDateTime,occurrenceString,period.start,period.end,serviceProvider.reference,serviceProvider.display,performedPeriod.start,performedPeriod.end,reaction,multipleBirthInteger,valueCodeableConcept.coding,valueCodeableConcept.text,hospitalization,hospitalization.dischargeDisposition.coding,hospitalization.dischargeDisposition.text,valueString,name.use,name.family,name.given,name.prefix,name.suffix,address.line,address.city,address.state,address.postalCode,address.country,given_names,street_address,full_address,full_name
2776,1,Patient,1,1,"[{'system': 'phone', 'value': '555-137-8673', 'use': 'home'}]",female,1983-05-03,False,"[{'language': {'coding': [{'system': 'urn:ietf:bcp:47', 'code': 'en-US', 'display': 'English (United States)'}], 'text': 'English (United States)'}}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,official,Doyle,Melany Renda,[Mrs.],NaN,776 Bergstrom Wall,Waltham,Massachusetts,02451,US,Melany Renda,776 Bergstrom Wall,"776 Bergstrom Wall, 02451 Waltham",Melany Renda Doyle
2777,1,Patient,1,1,"[{'system': 'phone', 'value': '555-137-8673', 'use': 'home'}]",female,1983-05-03,False,"[{'language': {'coding': [{'system': 'urn:ietf:bcp:47', 'code': 'en-US', 'display': 'English (United States)'}], 'text': 'English (United States)'}}]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,maiden,Tremblay,None,[Mrs.],NaN,776 Bergstrom Wall,Waltham,Massachusetts,02451,US,None,776 Bergstrom Wall,"776 Bergstrom Wall, 02451 Waltham",NaN
6507,1,Patient,1,1,"[{'system': 'phone', 'value': '555-137-8673', 'use': 'home'}]",None,1983-05-03,False,"[{'language': {'coding': [{'system': 'urn:ietf:bcp:47', 'code': 'en-US', 'display': 'English (United States)'}], 'text': 'English (United States)'}}]","[{'system': 'http://terminology.hl7.org/CodeSystem/v3-MaritalStatus', 'code': 'M', 'display': 'Married'}]",Married,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,official,Doyle,Melany Renda,[Mrs.],NaN,776 Bergstrom Wall,Waltham,Massachusetts,02451,US,Melany Renda,776 Bergstrom Wall,"776 Bergstrom Wall, 02451 Waltham",Melany Renda Doyle
6508,1,Patient,1,1,"[{'system': 'phone', 'value': '555-137-8673', 'use': 'home'}]",None,1983-05-03,False,"[{'language': {'coding': [{'system': 'urn:ietf:bcp:47', 'code': 'en-US', 'display': 'English (United States)'}], 'text': 'English (United States)'}}]","[{'system': 'http://terminology.hl7.org/CodeSystem/v3-MaritalStatus', 'code': 'M', 'display': 'Married'}]",Married,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,maiden,Tremblay,Melany Renda,[Mrs.],NaN,77

In [79]:
# Group by full_address and list the resourceType/id pairs
grouped = (
    df_addr_dups
      .groupby("full_address")
      .apply(lambda g: list(zip(g["resourceType"], g["resource_id"])))
)

for addr, resources in grouped.items():
    print(f"\n{addr} (seen {len(resources)} times):")
    for rtype, rid in resources:
        print(f"  - {rtype}/{rid}")



1006 Block Heights Unit 26, 00000 Mansfield (seen 2 times):
  - Patient/1
  - Patient/1

1006 Reynolds Pathway Unit 63, 02638 Dennis (seen 2 times):
  - Patient/1
  - Patient/1

101 Howell Branch, 01540 Oxford (seen 2 times):
  - Patient/1
  - Patient/1

1010 Prohaska Row Suite 24, 01469 Townsend (seen 2 times):
  - Patient/1
  - Patient/1

1011 Hartmann Lane, 01230 Great Barrington (seen 2 times):
  - Patient/1
  - Patient/1

1011 Ratke Way, 02364 Kingston (seen 2 times):
  - Patient/1
  - Patient/1

1012 DuBuque Mission Suite 38, 00000 Grafton (seen 2 times):
  - Patient/1
  - Patient/1

1016 Heathcote Mall, 00000 Billerica (seen 2 times):
  - Patient/1
  - Patient/1

1017 Legros Route, 02748 New Bedford (seen 2 times):
  - Patient/1
  - Patient/1

1018 Borer Vista Unit 1, 02148 Malden (seen 2 times):
  - Patient/1
  - Patient/1

1018 Kemmer Rapid Apt 41, 02467 Brookline (seen 2 times):
  - Patient/1
  - Patient/1

1022 Beahan Lodge, 02052 Medfield (seen 2 times):
  - Patient/1
  - 

C:\Users\Dee\AppData\Local\Temp\ipykernel_30380\4236009442.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: list(zip(g["resourceType"], g["resource_id"])))


In [81]:
# exact match
addr = "101 Howell Branch, 01540 Oxford"
df_oxford = df_clean[df_clean["full_address"] == addr]


df_oxford

,bundle_id,resourceType,resource_id,id,telecom,gender,birthDate,multipleBirthBoolean,communication,maritalStatus.coding,maritalStatus.text,status,type,participant,period,serviceProvider,class.system,class.code,subject.reference,subject.display,clinicalStatus,verificationStatus,code,onsetDateTime,recordedDate,clinicalStatus.coding,verificationStatus.coding,code.coding,code.text,category,effectiveDateTime,issued,valueQuantity,occurrenceDateTime,primarySource,vaccineCode.coding,vaccineCode.text,patient.reference,encounter.reference,maritalStatus,deceasedDateTime,valueQuantity.value,valueQuantity.unit,valueQuantity.system,valueQuantity.code,component,valueCodeableConcept,criticality,performedPeriod,reasonReference,active,reasonCode,abatementDateTime,occurrenceString,period.start,period.end,serviceProvider.reference,serviceProvider.display,performedPeriod.start,performedPeriod.end,reaction,multipleBirthInteger,valueCodeableConcept.coding,valueCodeableConcept.text,hospitalization,hospitalization.dischargeDisposition.coding,hospitalization.dischargeDisposition.text,valueString,name.use,name.family,name.given,name.prefix,name.suffix,address.line,address.city,address.state,address.postalCode,address.country,given_names,street_address,full_address,full_name
3334,1,Patient,1,1,"[{'system': 'phone', 'value': '555-225-3716', 'use': 'home'}]",None,1972-09-21,None,None,"[{'system': 'http://terminology.hl7.org/CodeSystem/v3-MaritalStatus', 'code': 'M', 'display': 'Married'}]",Married,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,official,Kessler,Marquitta Lavonda,[Mrs.],NaN,101 Howell Branch,Oxford,Massachusetts,01540,US,Marquitta Lavonda,101 Howell Branch,"101 Howell Branch, 01540 Oxford",Marquitta Lavonda Kessler
3335,1,Patient,1,1,"[{'system': 'phone', 'value': '555-225-3716', 'use': 'home'}]",None,1972-09-21,None,None,"[{'system': 'http://terminology.hl7.org/CodeSystem/v3-MaritalStatus', 'code': 'M', 'display': 'Married'}]",Married,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,maiden,Murray,Marquitta Lavonda,[Mrs.],NaN,101 Howell Branch,Oxford,Massachusetts,01540,US,Marquitta Lavonda,101 Howell Branch,"101 Howell Branch, 01540 Oxford",Marquitta Lavonda Murray


In [26]:
import json
import polars as pl

In [65]:
# 1) Keep only one row per unique address
unique_addr = df_clean.drop_duplicates(subset="full_address")

# 2) List all states
states = sorted(unique_addr["address.state"].dropna().unique())
print("States in dataset:")
for s in states:
    print(f" - {s}")

# 3) Count unique states, cities, and postal codes
num_states       = len(states)
num_cities       = unique_addr["address.city"].dropna().nunique()
num_postal_codes = unique_addr["address.postalCode"].dropna().nunique()

print(f"\nNumber of unique states:     {num_states}")
print(f"Number of unique cities:     {num_cities}")
print(f"Number of unique post codes: {num_postal_codes}")


States in dataset:
 - MA
 - Massachusetts

Number of unique states:     2
Number of unique cities:     391
Number of unique post codes: 484


In [66]:
# 1) Dedupe on the already-built full_address
unique_addr = df_clean.drop_duplicates(subset="full_address")

# 2) Normalize city and postalCode for counting
cities_norm       = unique_addr["address.city"].dropna().str.strip().str.lower()
postal_codes_norm = unique_addr["address.postalCode"].dropna().str.strip().str.lower()

# 3) Get unique (case-insensitive) lists
states = sorted(unique_addr["address.state"].dropna().unique())  # states usually already standardized
cities = sorted(cities_norm.unique())
zips   = sorted(postal_codes_norm.unique())

# 4) Print
print("States in dataset:")
for s in states:
    print(f" - {s}")

print(f"\nNumber of unique states:     {len(states)}")
print(f"Number of unique cities:     {len(cities)}")
print(f"Number of unique post codes: {len(zips)}")


States in dataset:
 - MA
 - Massachusetts

Number of unique states:     2
Number of unique cities:     300
Number of unique post codes: 484


In [69]:
print(df_clean.columns)

Index(['bundle_id', 'resourceType', 'resource_id', 'id', 'telecom', 'gender', 'birthDate', 'multipleBirthBoolean', 'communication', 'maritalStatus.coding', 'maritalStatus.text', 'status', 'type', 'participant', 'period', 'serviceProvider', 'class.system', 'class.code', 'subject.reference', 'subject.display', 'clinicalStatus', 'verificationStatus', 'code', 'onsetDateTime', 'recordedDate', 'clinicalStatus.coding', 'verificationStatus.coding', 'code.coding', 'code.text', 'category', 'effectiveDateTime', 'issued', 'valueQuantity', 'occurrenceDateTime', 'primarySource', 'vaccineCode.coding', 'vaccineCode.text', 'patient.reference', 'encounter.reference', 'maritalStatus', 'deceasedDateTime', 'valueQuantity.value', 'valueQuantity.unit', 'valueQuantity.system', 'valueQuantity.code', 'component', 'valueCodeableConcept', 'criticality', 'performedPeriod', 'reasonReference', 'active', 'reasonCode', 'abatementDateTime', 'occurrenceString', 'period.start', 'period.end', 'serviceProvider.reference',


In [82]:
category_counts = df_clean["resourceType"].value_counts()
print(category_counts)

resourceType
Patient               3435
Encounter             2062
Observation            902
Procedure              893
Condition              786
Practitioner           698
Organization           655
Immunization           437
AllergyIntolerance     365
Name: count, dtype: int64


In [85]:
import json
import pandas as pd

# 1) Load your array of Bundles
with open("./data/fhir_v2.jsonl", "r", encoding="utf-8") as f:
    bundles = json.load(f)
# 2) Build rows: one dict per Bundle
rows = []
for b in bundles:
    row = {
        "bundle_id":   b["id"],
        "bundle_type": b.get("type"),
    }
    # collect IDs by resourceType
    for entry in b.get("entry", []):
        r = entry["resource"]
        col = f"{r['resourceType'].lower()}_ids"
        row.setdefault(col, []).append(r["id"])
    rows.append(row)

# 3) Create the DataFrame
df_bundles = pd.DataFrame(rows)

# 4) Ensure every *_ids column is a list (not NaN)
for col in df_bundles.columns:
    if col.endswith("_ids"):
        df_bundles[col] = df_bundles[col].apply(lambda x: x if isinstance(x, list) else [])


In [ ]:
import json
import pandas as pd

# suppose bundles is a list of Bundle‐dicts
with open('./data/fhir_v2.jsonl', 'r', encoding='utf-8') as f:
    bundles = json.load(f)

records = []
for bundle_idx, bundle in enumerate(bundles, start=1):
    for entry in bundle.get("entry", []):
        r = entry.get("resource", {})
        rec = {
            "bundle_id": bundle_idx,            # use enumerate index
            "resourceType": r.get("resourceType"),
            "resource_id":   r.get("id"),
            **r
        }
        records.append(rec)

df_bundles = pd.json_normalize(records)

# Thanh759.Weber641@example.com



In [90]:
df_bundles.head(n=10)

,bundle_id,resourceType,resource_id,id,name,telecom,gender,birthDate,address,multipleBirthBoolean,communication,maritalStatus.coding,maritalStatus.text,status,type,participant,period,serviceProvider,class.system,class.code,subject.reference,subject.display,clinicalStatus,verificationStatus,code,onsetDateTime,recordedDate,clinicalStatus.coding,verificationStatus.coding,code.coding,code.text,category,effectiveDateTime,issued,valueQuantity,occurrenceDateTime,primarySource,vaccineCode.coding,vaccineCode.text,patient.reference,encounter.reference,maritalStatus,deceasedDateTime,valueQuantity.value,valueQuantity.unit,valueQuantity.system,valueQuantity.code,component,valueCodeableConcept,criticality,performedPeriod,reasonReference,active,reasonCode,abatementDateTime,occurrenceString,period.start,period.end,serviceProvider.reference,serviceProvider.display,performedPeriod.start,performedPeriod.end,reaction,multipleBirthInteger,valueCodeableConcept.coding,valueCodeableConcept.text,hospitalization,hospitalization.dischargeDisposition.coding,hospitalization.dischargeDisposition.text,valueString
0,1,Patient,1,1,"[{'use': 'official', 'family': 'Corkery', 'given': ['Rudolf', 'Nolan'], 'prefix': ['Mr.']}]","[{'system': 'phone', 'value': '555-124-9334', 'use': 'home'}]",male,None,"[{'line': ['983 Gutmann Burg'], 'city': 'Boston', 'state': 'Massachusetts', 'postalCode': '02111', 'country': 'US'}]",False,"[{'language': {'coding': [{'system': 'urn:ietf:bcp:47', 'code': 'en-US', 'display': 'English (United States)'}], 'text': 'English (United States)'}}]","[{'system': 'http://terminology.hl7.org/CodeSystem/v3-MaritalStatus', 'code': 'M', 'display': 'Married'}]",Married,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Encounter,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unknown,None,None,NaN,NaN,http://terminology.hl7.org/CodeSystem/v3-ActCode,AMB,Patient/1,Mr. Rudolf Nolan Corkery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Patient,1,1,"[{'use': 'official', 'family': 'Bashirian', 'given': ['Hobert', 'Armand'], 'prefix': ['Mr.']}]",None,None,1988-11-12,"[{'line': ['767 Windler Well'], 'city': 'Easthampton', 'state': 'Massachusetts', 'postalCode': '01027', 'country': 'US'}]",False,"[{'language': {'coding': [{'system': 'urn:ietf:bcp:47', 'code': 'en-US', 'display': 'English (United States)'}], 'text': 'English (United States)'}}]","[{'system': 'http://terminology.hl7.org/CodeSystem/v3-MaritalStatus', 'code': 'W', 'display': 'Widowed'}]",Widowed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,Condition,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Patient/1,NaN,NaN,NaN,NaN,2014-02-03T00:38:02+01:00,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,Condition,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Patient/1,NaN,NaN,NaN,NaN,2008-01-12T01:09:00+01:00,None,"[{'system': 'http://terminology.hl7.org/CodeSystem/condition-clinical', 'code': 'active'}]","[{'system': 'http://terminology.hl7.org/CodeSystem/condition-ver-status', 'code': 'confirmed'}]","[{'system': 'http://snomed.info/sct', 'code': '706893006', 'display': 'Victim of intimate partner abuse (finding)'}]",Victim of intimate partner abuse (finding),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,Observation,